In [1]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import cross_val_score, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.impute import KNNImputer

In [3]:
df1 = pd.read_csv('COHORT_1_data.csv',usecols=['PATNO', 'SDMTOTAL', 'STAI_TOT', 'SFT_TOT', 'SCOPA_AUT_TOT',
       'REMSLEEP_TOT', 'QUIP_A', 'QUIP_B', 'QUIP_C', 'QUIP_D', 'QUIP_E',
       'MoCA_score', 'LNS_TOT', 'HVLT_TOT_Recall', 'HVLT_DCR_REC',
       'HVLT_RETENTION', 'GDS_TOT', 'GDS_Depressed', 'ESS_TOT', 'ESS_Sleepy',
       'BJLOT_TOT', 'COHORT'])
df2 = pd.read_csv('COHORT_2_data.csv',usecols=['PATNO', 'SDMTOTAL', 'STAI_TOT', 'SFT_TOT', 'SCOPA_AUT_TOT',
       'REMSLEEP_TOT', 'QUIP_A', 'QUIP_B', 'QUIP_C', 'QUIP_D', 'QUIP_E',
       'MoCA_score', 'LNS_TOT', 'HVLT_TOT_Recall', 'HVLT_DCR_REC',
       'HVLT_RETENTION', 'GDS_TOT', 'GDS_Depressed', 'ESS_TOT', 'ESS_Sleepy',
       'BJLOT_TOT', 'COHORT'])
df3 = pd.read_csv('COHORT_3_data.csv',usecols=['PATNO', 'SDMTOTAL', 'STAI_TOT', 'SFT_TOT', 'SCOPA_AUT_TOT',
       'REMSLEEP_TOT', 'QUIP_A', 'QUIP_B', 'QUIP_C', 'QUIP_D', 'QUIP_E',
       'MoCA_score', 'LNS_TOT', 'HVLT_TOT_Recall', 'HVLT_DCR_REC',
       'HVLT_RETENTION', 'GDS_TOT', 'GDS_Depressed', 'ESS_TOT', 'ESS_Sleepy',
       'BJLOT_TOT', 'COHORT'])
df4 = pd.read_csv('COHORT_4_data.csv',usecols=['PATNO', 'SDMTOTAL', 'STAI_TOT', 'SFT_TOT', 'SCOPA_AUT_TOT',
       'REMSLEEP_TOT', 'QUIP_A', 'QUIP_B', 'QUIP_C', 'QUIP_D', 'QUIP_E',
       'MoCA_score', 'LNS_TOT', 'HVLT_TOT_Recall', 'HVLT_DCR_REC',
       'HVLT_RETENTION', 'GDS_TOT', 'GDS_Depressed', 'ESS_TOT', 'ESS_Sleepy',
       'BJLOT_TOT', 'COHORT'])

In [4]:
df3.columns

Index(['PATNO', 'SDMTOTAL', 'STAI_TOT', 'SFT_TOT', 'SCOPA_AUT_TOT',
       'REMSLEEP_TOT', 'QUIP_A', 'QUIP_B', 'QUIP_C', 'QUIP_D', 'QUIP_E',
       'MoCA_score', 'LNS_TOT', 'HVLT_TOT_Recall', 'HVLT_DCR_REC',
       'HVLT_RETENTION', 'GDS_TOT', 'GDS_Depressed', 'ESS_TOT', 'ESS_Sleepy',
       'BJLOT_TOT', 'COHORT'],
      dtype='object')

In [5]:
# Define a function to fill missing values and apply KNN imputation
def fill_missing_and_impute(df):
    # Group DataFrame by 'PATNO' and fill missing values with median within each group
    grouped_df = df.groupby('PATNO').transform(lambda x: x.fillna(x.median()))

    # Add 'PATNO' column back to the filled DataFrame
    df_filled = pd.concat([df['PATNO'], grouped_df], axis=1)

    # Calculate the mean of each feature after filling missing values
    grouped_mean_df = df_filled.groupby('PATNO').mean().reset_index()

    # Apply KNN imputation to handle remaining missing values
    knn = KNNImputer()
    grouped_mean_df = pd.DataFrame(knn.fit_transform(grouped_mean_df), columns=grouped_mean_df.columns)
    
    return grouped_mean_df

# Apply the function to all four dataframes
df1_filled = fill_missing_and_impute(df1)
df2_filled = fill_missing_and_impute(df2)
df3_filled = fill_missing_and_impute(df3)
df4_filled = fill_missing_and_impute(df4)

c:\Users\91881\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\91881\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\91881\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\91881\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\91881\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\9

In [6]:
f = pd.concat([df1_filled,df2_filled,df3_filled,df4_filled])

In [7]:
le = LabelEncoder()
f['COHORT'] = le.fit_transform(f['COHORT'])

In [8]:
f.sample(10)

,PATNO,SDMTOTAL,STAI_TOT,SFT_TOT,SCOPA_AUT_TOT,REMSLEEP_TOT,QUIP_A,QUIP_B,QUIP_C,QUIP_D,...,LNS_TOT,HVLT_TOT_Recall,HVLT_DCR_REC,HVLT_RETENTION,GDS_TOT,GDS_Depressed,ESS_TOT,ESS_Sleepy,BJLOT_TOT,COHORT
320,3770.0,33.192308,106.230769,46.346154,18.769231,6.269231,0.0,0.0,0.076923,0.076923,...,9.961538,21.423077,9.076923,0.760068,8.076923,0.923077,12.000000,0.769231,12.346154,0
1627,250804.0,48.000000,46.000000,21.000000,16.000000,9.000000,0.0,0.0,0.000000,0.000000,...,13.000000,30.000000,12.000000,1.000000,1.000000,0.000000,3.000000,0.000000,13.000000,3
649,42418.0,36.333333,91.750000,25.666667,13.250000,1.166667,0.0,0.0,0.000000,0.000000,...,6.500000,21.000000,10.916667,0.834806,7.166667,0.916667,8.500000,0.166667,14.166667,0
117,3478.0,31.333333,50.333333,33.500000,4.333333,2.666667,0.0,0.0,0.000000,0.000000,...,9.333333,19.500000,10.000000,0.867063,0.166667,0.000000,4.000000,0.000000,11.333333,1
805,101025.0,23.800000,83.600000,17.800000,27.800000,1.200000,0.0,0.0,0.000000,0.000000,...,6.400000,16.600000,7.200000,0.500000,3.600000,0.000000,3.600000,0.000000,13.600000,0
709,111459.0,47.000000,75.000000,19.000000,18.000000,12.000000,0.0,0.0,0.000000,0.000000,...,8.000000,14.000000,11.000000,1.000000,3.000000,0.000000,12.000000,1.000000,14.000000,3
92,3218.0,13.958333,74.083333,24.708333,11.250000,8.291667,0.0,0.0,0.000000,0.000000,...,6.125000,16.708333,8.291667,0.783482,2.875000,0.250000,6.291667,0.083333,4.625000,0
62,3864.0,32.880000,65.420000,37.920000,18.240000,5.300000,0.0,0.0,0.040000,0.120000,...,7.120000,21.040000,9.680000,0.727980,2.100000,0.120000,9.240000,0.440000,12.520000,2
592,41384.0,50.055556,65.166667,42.000000,10.277778,1.333333,0.0,0.0,0.000000,0.000000,...,9.944444,30.333333,11.777778,1.028114,0.833333,0.000000,7.666667,0.055556,13.611111,0
1452,228473.0,44.600000,52.800000,16.000000,14.400000,5.600000,0.0,0.0,0.200000,0.200000,...,9.000000,21.200000,8.000000,0.543333,2.000000,0.200000,3.200000,0.000000,13.200000,3


In [9]:
# Convert all columns from float to integer
f = f.apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

In [10]:
f.sample(5)

,PATNO,SDMTOTAL,STAI_TOT,SFT_TOT,SCOPA_AUT_TOT,REMSLEEP_TOT,QUIP_A,QUIP_B,QUIP_C,QUIP_D,...,LNS_TOT,HVLT_TOT_Recall,HVLT_DCR_REC,HVLT_RETENTION,GDS_TOT,GDS_Depressed,ESS_TOT,ESS_Sleepy,BJLOT_TOT,COHORT
1339,215173,26,69,19,21,13,0,1,0,0,...,11,16,7,0,3,0,5,0,12,3
1548,241189,34,67,22,33,6,0,0,0,0,...,6,22,11,0,5,1,3,0,7,0
469,4125,36,45,22,6,4,0,0,0,0,...,10,17,9,0,0,0,5,0,13,0
26,3066,39,75,55,12,5,0,0,0,0,...,11,21,9,0,2,0,6,0,13,0
749,59507,44,99,38,35,7,0,0,0,0,...,9,26,11,0,5,0,16,1,13,0


In [11]:
mask = f['COHORT'] == 2
df = f[~mask]

In [12]:
from imblearn.over_sampling import RandomOverSampler
# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority')

In [13]:
X =  f.drop(['COHORT'],axis=1)
y= f['COHORT']

X_train, X_test, y_train, y_test = train_test_split(X,y , 
                                   random_state=104,  
                                   test_size=0.25,  
                                   shuffle=True)

In [14]:
import pandas as pd

# Assuming f is a DataFrame
f.to_csv('non_motor_final.csv', index=False)


In [15]:
# fit and apply the transform
X_over, y_over = oversample.fit_resample(X, y)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_over,y_over , 
                                   random_state=104,  
                                   test_size=0.25,  
                                   shuffle=True)

In [17]:
svm = SVC(kernel="rbf", gamma="auto",probability=True)
svm.fit(X_train,y_train)
prediction = svm.predict(X_test)

accuracy_SVM = accuracy_score(prediction, y_test)
print(accuracy_SVM)

0.6364285714285715


In [18]:
dec = DecisionTreeClassifier(random_state=0)
dec.fit(X_train,y_train)
pred = dec.predict(X_test)
accuracy_DEC = accuracy_score(pred, y_test)
print(accuracy_DEC)

0.7557142857142857


In [19]:
rand = RandomForestClassifier(max_depth=15, random_state=0)
rand.fit(X_train,y_train)
rand_pred = rand.predict(X_test)
rand_accuracy = accuracy_score(rand_pred,y_test)
print(rand_accuracy)

0.8214285714285714


In [20]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train,y_train)
neigh_pred = neigh.predict(X_test)
neigh_accuracy = accuracy_score(neigh_pred,y_test)
print(neigh_accuracy)

0.7535714285714286


In [21]:
pip install tensorflow


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Assuming X_train, X_test, y_train, and y_test are already defined

# Initialize and train the gradient boosting classifier
gb_clf = GradientBoostingClassifier()
gb_clf.fit(X_train, y_train)

# Make predictions on the test set
gb_pred = gb_clf.predict(X_test)

# Calculate accuracy
gb_accuracy = accuracy_score(gb_pred, y_test)
print("Gradient Boosting Classifier Accuracy:", gb_accuracy)

Gradient Boosting Classifier Accuracy: 0.8035714285714286


In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
# Make predictions on the test set
logreg_pred = logreg.predict(X_test)

# Calculate accuracy
logreg_accuracy = accuracy_score(logreg_pred, y_test)
print("Logistic Regression Accuracy:", logreg_accuracy)

Logistic Regression Accuracy: 0.6021428571428571


c:\Users\91881\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Assuming X_train, X_test, y_train, and y_test are already defined

# Initialize and train the linear regression model
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# Make predictions on the test set
lin_reg_pred = lin_reg.predict(X_test)

# Calculate mean squared error
mse = mean_squared_error(y_test, lin_reg_pred)
print("Mean Squared Error:", mse)
predicted_classes = np.round(lin_reg_pred)

# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_classes)
print("Accuracy:", accuracy)

Mean Squared Error: 1.40780897163303
Accuracy: 0.22214285714285714


In [25]:
from sklearn.ensemble import VotingClassifier
# Create a Soft Voting Classifier
labels = ['Decision Forest', 'Random Forest', 'GB']
voting_clf_hard = VotingClassifier(
	estimators=[
		(labels[1], rand), # Include the second classifier (Random Forest)
		(labels[2],gb_clf ), # Include the third classifier (Naive Bayes)
	],
	voting='hard' # Specify soft voting, where class probabilities are combined
)


In [26]:
voting_clf_hard.fit(X_train,y_train)
V_soft_pred = voting_clf_hard.predict(X_test)
voting_soft_accuracy = accuracy_score(V_soft_pred,y_test)
print(voting_soft_accuracy)

0.8207142857142857


In [27]:
pip install joblib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
import pickle
import joblib
model=voting_clf_hard

with open("model_nonMotor.pkl", "wb") as f:
    pickle.dump(model, f)
    
with open("model_nonMotor.pkl", "rb") as f:
    model = pickle.load(f)
    
joblib.dump(model, "model_nonMotor.joblib")
model = joblib.load("model_nonMotor.joblib")